In [49]:
import pandas as pd 
import http.client
import os
import json
import time
from datetime import datetime

In [50]:
def main():
    pass

if __name__ == '__main__':
    main()

# Variables
access_token = os.environ.get('access_token_surveymonkey')
survey_id = 315864260
start_time = time.time()

# Functions
def parse_timestamp(text):
    d = text.replace('+00:00', '')
    return datetime.strptime(d, '%Y-%m-%dT%H:%M:%S').strftime('%Y-%m-%d')

def update_survey_details(survey_id, access_token):
    '''
    Calls the SurveyMonkey API to get the details of given survey, including the questions and general structure.
    survey_id = 12345678
    access_token = API access token from SurveyMonkey
    '''
    conn = http.client.HTTPSConnection("api.surveymonkey.com")
    headers = {
        'Accept': "application/json",
        'Authorization': f"Bearer {access_token}"
        }

    conn.request("GET", f"/v3/surveys/{survey_id}/details", headers=headers)
    res = conn.getresponse()
    survey_details = res.read()
    json_details = json.loads(survey_details)
        
    if res.status >= 400: 
        raise RuntimeError(f'Request failed: {res.status}')
    
    # # Write the JSON response to a file
    # with open('details.json', 'w') as f:
    #     data = json.dump(json_details, f, indent=4, sort_keys=True)

    return json_details

def update_survey_data(survey_id, access_token):
    '''
    Calls the SurveyMonkey API to get all of the data of a given survey.
    survey_id = 12345678
    access_token = API access token from SurveyMonkey
    '''
    conn = http.client.HTTPSConnection("api.surveymonkey.com")
    headers = {
        'Accept': "application/json",
        'Authorization': f"Bearer {access_token}"
        }

    conn.request("GET", f"/v3/surveys/{survey_id}/responses/bulk", headers=headers)
    res = conn.getresponse()
    survey_data = res.read()
    json_data = json.loads(survey_data)

    if res.status >= 400: 
        raise RuntimeError(f'Request failed: {res.status}')

    # # Write the JSON response to a file
    # with open('data.json', 'w') as f:
    #     data = json.dump(json_data, f, indent=4, sort_keys=True)
    
    return json_data

details = update_survey_details(access_token=access_token, survey_id=survey_id)
data = update_survey_data(access_token=access_token, survey_id=survey_id)

In [51]:
# Map Question ID to Question Text
question_id = pd.json_normalize(details['pages'], ['questions'])['id']
question_text = pd.json_normalize(details['pages'], ['questions', 'headings'])['heading']

questions = dict(zip(question_id, question_text))

# Pull response ID
response_id = pd.json_normalize(data['data'])['id']

# Map student ID to college variable
college_id = pd.json_normalize(data['data'])['custom_variables.College']
colleges = dict(zip(response_id, college_id))

# Map response ID to response date
response_dates = pd.json_normalize(data['data'])['date_created']
formatted_date = [parse_timestamp(i) for i in response_dates]
completed_date = dict(zip(response_id, formatted_date))

# Map student ID to semester variable
semester_value = pd.json_normalize(data['data'])['custom_variables.Semester']

semesters = dict(zip(response_id, semester_value))

# Map student ID to programme variable
program_value = pd.json_normalize(data['data'])['custom_variables.Program']

programs = dict(zip(response_id, program_value))

# Map ID to module variable
module_value = pd.json_normalize(data['data'])['custom_variables.Module']

modules = dict(zip(response_id, module_value))

In [52]:
# Building the dataframe
json_df = pd.json_normalize(data['data'], ['pages', 'questions', 'answers'], ['id', ['questions', 'answers', 'id']])
json_df['completion_date'] = json_df['id'].map(completed_date)
json_df['college'] = json_df['id'].map(colleges)
json_df['semester'] = json_df['id'].map(semesters)
json_df['program'] = json_df['id'].map(programs)
json_df['module'] = json_df['id'].map(modules)
json_df['question'] = json_df['questions.answers.id'].map(questions)

# Rename columns
json_df.rename(columns={
    "choice_metadata.weight" : "answer",
    "questions.answers.id" : "question_id",
    "id" : "respondent_id"
}, inplace=True)

# Fill NA in the answer column
json_df['answer'].fillna(json_df['text'], inplace=True)

# Reshaping
json_df = json_df[[
    'respondent_id',
    'completion_date',
    'college',
    'semester',
    'program',
    'module',
    'question_id',
    'question',
    'answer'
]]

# Questions as headers - values / index / column headers
df = json_df.pivot_table('answer', ['respondent_id', 'completion_date', 'college', 'program', 'semester', 'module'], 'question', aggfunc=sum).fillna(0)
print(f'Script executed in: {time.time() - start_time} seconds')
df.head()

Script executed in: 2.176816940307617 seconds


,,,,,question,Any other comments you would like to make about your program?,"How likely is it that you would recommend the university to a friend? (1 - Least likely, 10 - Most likely)",I felt the lecturer(s) had a good knowledge of the subject,Learning resources were easily available for this module,"Overall I am satisfied with the quality of my Facilities and Virtual Learning Environment (1 - Least likely, 10 - Most likely)","Overall I am satisfied with the quality of my Lecturer/s (1 - Least likely, 10 - Most likely)","Overall I am satisfied with the quality of my Programme (1 - Least likely, 10 - Most likely)",Overall I felt I understood the module content,The expectations of the assessments were made clear to me,The expectations of the module were clear,The facilities and online resources used met the requirements of the subject,The lecturer(s) explained concepts clearly and helped me understand the content of the module,The lecturer(s) gave me helpful feedback on my work,The lecturer(s) often related the content to real life examples,The lecturer(s) provided a positive learning environment,The lecturer(s) was reasonably available for consultation outside of class time,The workload was appropriate for this module,This module was relevant to my future career,What did you like most about this trimester?,What would you change about this trimester?
respondent_id,completion_date,college,program,semester,module,,,,,,,,,,,,,,,,,,,,
13270572817,2022-01-19,BPC,Audio,202103,MPP01,fgfgfgf,7,2,3,5,5,6,3,3,3,3,3,4,3,2,2,3,4,fgfgfgfg,fggfgf
13270668324,2022-01-19,BPC,Audio,202103,MPP01,jkjkjkjkj,6,0,4,6,6,5,4,3,3,3,2,3,2,3,3,4,4,jkjkjkjk,jkjkjkjkjkj
13270725773,2022-01-19,ARUC,Film,202201,MPP02,ghghghghg,8,4,3,7,7,7,2,3,3,3,3,2,3,3,4,4,4,ghghghgh,ghghghghg
13270727929,2022-01-19,TPC,Web,202202,MPP03,fgfgfgfgf,5,3,3,6,5,6,3,2,3,3,3,3,2,2,2,2,3,fgfgfgf,fgfgfg
